In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('diabetes.csv')

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [ ]:
X.shape

(768, 8)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size= 0.2,random_state=1)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [ ]:
model = Sequential()
model.add(Dense(32,activation="relu",input_dim=8))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="Adam", loss="binary_crossentropy", metrics="accuracy")

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 14ms/step - loss: 0.7221 - accuracy: 0.5603 - val_loss: 0.6699 - val_accuracy: 0.6364
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6589 - accuracy: 0.6694 - val_loss: 0.6202 - val_accuracy: 0.6818
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6121 - accuracy: 0.7085 - val_loss: 0.5827 - val_accuracy: 0.7078
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5773 - accuracy: 0.7313 - val_loss: 0.5523 - val_accuracy: 0.7597
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5495 - accuracy: 0.7459 - val_loss: 0.5304 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5271 - accuracy: 0.7573 - val_loss: 0.5154 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5118 - accuracy: 0.7704 - val_loss: 0.5026 - val_accuracy: 0.7922
Epoch 8/100


How to automate to find best parameter
1. How to select appropriate optimizer
2. To select no. of nodes
3. How to select no. of layers
4. All in one model

In [ ]:
pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
#Selecting Best Optimizer

def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation="relu",input_dim=8))
  model.add(Dense(1,activation="sigmoid"))

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics="accuracy")

  return model

In [ ]:
#Creating tuner

tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5)

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
#Extracting best model from 4 of them
model = tuner.get_best_models()[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Now training on particular best model
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 3s 29ms/step - loss: 0.5204 - accuracy: 0.7541 - val_loss: 0.5012 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5088 - accuracy: 0.7557 - val_loss: 0.4934 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4978 - accuracy: 0.7671 - val_loss: 0.4872 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4899 - accuracy: 0.7655 - val_loss: 0.4831 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 16ms/step - loss: 0.4826 - accuracy: 0.7655 - val_loss: 0.4793 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4775 - accuracy: 0.7655 - val_loss: 0.4767 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4739 - accuracy: 0.7671 - val_loss: 0.4747 - val_accuracy: 0.7922
Epoch 1

In [ ]:
#Selecting the no. of nuerons in each layer

def build_model(hp):
  model = Sequential()
  units = hp.Int('units',8,128) #we can also add step like "step=8" withnin brackets

  model.add(Dense(units=units,activation="relu",input_dim=8))
  model.add(Dense(1,activation="sigmoid"))


  model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics="accuracy")

  return model

In [ ]:
#Creating tuner

tnr = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                      directory = 'mydir',
                      project_name ='demo')

In [ ]:
tnr.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [ ]:
tnr.get_best_hyperparameters()[0].values

{'units': 87}

In [ ]:
#Extracting best model from with respectively best no. of units
model = tnr.get_best_models()[0]

In [ ]:
#Now training on particular best model
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 16ms/step - loss: 0.4956 - accuracy: 0.7622 - val_loss: 0.4896 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4829 - accuracy: 0.7704 - val_loss: 0.4791 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4741 - accuracy: 0.7752 - val_loss: 0.4723 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4675 - accuracy: 0.7736 - val_loss: 0.4692 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4639 - accuracy: 0.7785 - val_loss: 0.4642 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4605 - accuracy: 0.7736 - val_loss: 0.4633 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4575 - accuracy: 0.7785 - val_loss: 0.4624 - val_accuracy: 0.7987
Epoch 14

In [ ]:
#Selecting no. of layers

def build_model(hp):
  model = Sequential()
  model.add(Dense(87,activation="relu",input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(87,activation='relu'))
  model.add(Dense(1,activation="sigmoid"))

  model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics="accuracy")

  return model

In [ ]:
#Creating tuner

tnr = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                      directory = 'mydir',
                      project_name ='num_layer')

In [ ]:
tnr.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 18s


In [ ]:
tnr.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [ ]:
#All at once

def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',1,10)):

    if counter ==0:
      model.add(
          Dense(
              hp.Int('units'+str(i),8,128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
              input_dim = 8
            )
          )
      #adding dropout layer
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(
          Dense(
              hp.Int('units'+str(i),8,128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
              input_dim = 8
            )
          )
      #adding dropout layer
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter += 1

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer = hp.Choice('optimizer', values=['adam','rmsprop','adagrad','adadelta','sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [ ]:
#Creating tuner

tnr = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                      directory = 'mydir',
                      project_name ='final1')

In [ ]:
tnr.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6688311696052551

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 11s


In [ ]:
tnr.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 56,
 'activation0': 'tanh',
 'dropout0': 0.9,
 'optimizer': 'rmsprop'}